In [ ]:
import os
import re
import ast
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
from nltk.tokenize import RegexpTokenizer
from preprocess import preprocess
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

train_data = pd.read_csv(os.path.join("data", "train.csv"))
test_data = pd.read_csv(os.path.join("data", "test.csv"))
train_data.dropna(inplace=True)
X, y = train_data.drop(["price"], axis=1), train_data["price"]
y = y.to_numpy().astype(int)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)
X_test = test_data

X_train = preprocess(X_train).to_numpy().astype(float)
X_val = preprocess(X_val).to_numpy().astype(float)
X_test = preprocess(X_test)

# scale
scaler =  RobustScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

X_train.shape

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators' : [10, 30, 50, 70],
    'learning_rate' : [0.01, 0.03, 0.1, 0.3]
}

# Adaboost with DecisionTree as base estimator
ada_clf = AdaBoostClassifier(
    estimator=DecisionTreeClassifier()
)

gscv = GridSearchCV(ada_clf, param_grid, cv=5, scoring="f1")
gscv.fit(X_train, y_train)
ada_clf = gscv.best_estimator_
# ada_test_accuracy = ada_clf.score(X_val, y_val)
# print("Adaboost Test Accuracy:", ada_test_accuracy)